In [54]:
import requests
import datetime
import pandas as pd
import xml.etree.ElementTree as ET

pd.options.plotting.backend = "plotly"

def ParseDateTime(date, separator = "."):
    divide = date.split(separator)
    
    try:
        return datetime.datetime(int(divide[2]), int(divide[1]), int(divide[0]))
    except ValueError:
        print("Такой даты не существует!")
        raise
    except IndexError:
        print("Некректный формат даты! Должно быть: DD/MM/YYYY")
        raise

def GetValutaCodes():
    url = "https://cbr.ru/scripts/XML_val.asp?d=0"

    response = requests.post(url)

    if response.status_code != 200:
        print("Не удалось подключиться к сервису центробанка!")
        raise Exception
    else:
        root = ET.fromstring(response.content)

        result = {}

        for entry in root.findall("Item"):
            result[entry.find("ParentCode").text.replace(" ", "")] = entry.find("Name").text

        return result

    

valutasCodes = GetValutaCodes()

print("Введите начальную дату! Формат = DD/MM/YYYY")
dateFrom = input()
dateFromInType = ParseDateTime(dateFrom, separator= "/")

print("Введите конечную дату! Формат = DD/MM/YYYY")
dateTo = input()
dateToInType = ParseDateTime(dateTo, separator= "/")

print(f"Введите код валюты! Допустимые:\n{valutasCodes}")
valutaCode = input()

if dateToInType <= dateFromInType:
    print("Конечная дата не может быть меньше или равной начальной!")
else:
    valuta = valutasCodes.get(valutaCode)

    if valuta == None:
        print(f"{valutaCode} - не существует валюты с таким кодом!")
    else:
        url = f"https://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={dateFrom}&date_req2={dateTo}&VAL_NM_RQ={valutaCode}"

        response = requests.post(url)

        if response.status_code != 200:
            print("Не удалось подключиться к сервису центробанка!")
        else:
            root = ET.fromstring(response.content)

            rawData = {}

            for entry in root.findall("Record"):
                rawData[ParseDateTime(entry.attrib["Date"])] = float(entry.find('Value').text.replace(',', '.')) / int(entry.find("Nominal").text)

            dataFrame = pd.DataFrame.from_dict(rawData, orient= "index", columns= [valuta])

            graph = dataFrame.plot(title=f"Российский рубль - {valuta}", template= "plotly_dark", 
                                   labels=dict(index="Дата", value="Курс", variable="Валюта"))
            
            graph.update_yaxes(tickprefix="₽")
            graph.show()


Введите начальную дату! Формат = DD/MM/YYYY
Некректный формат даты! Должно быть: DD/MM/YYYY


IndexError: list index out of range